In [1]:
from qiskit.circuit.library import StatePreparation
from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, AncillaRegister

In [2]:
def prepare_state_generic(state_dict, num_of_qubits):
    coefficients = [state_dict.get(format(i, '0' + str(num_of_qubits) + 'b'), 0) for i in range(2**num_of_qubits)]
    state = StatePreparation(coefficients)
    return state

def prepare_circuit_initial_state(value, probability):
    zero_state_dict = {
        '00000': 1/4,
        '10010': 1/4,
        '01001': 1/4,
        '10100': 1/4,
        '01010': 1/4,
        '11011': -1/4,
        '00110': -1/4,
        '11000': -1/4,
        '11101': -1/4,
        '00011': -1/4,
        '11110': -1/4,
        '01111': -1/4,
        '10001': -1/4,
        '01100': -1/4,
        '10111': -1/4,
        '00101': 1/4
    }

    one_state_dict = {
        '11111': 1/4,
        '01101': 1/4,
        '10110': 1/4,
        '01011': 1/4,
        '10101': 1/4,
        '00100': -1/4,
        '11001': -1/4,
        '00111': -1/4,
        '00010': -1/4,
        '11100': -1/4,
        '00001': -1/4,
        '10000': -1/4,
        '01110': -1/4,
        '10011': -1/4,
        '01000': -1/4,
        '11010': 1/4
    }

    initial_state = None
    if value == True:
        initial_state = one_state_dict
    else:
        initial_state = zero_state_dict

    # Prepare state according to the input 
    state_preparation = prepare_state_generic(initial_state, 5)

    return state_preparation

In [3]:
def five_qubit_error_code_circuit(value, probability):
    qc = QuantumCircuit()
    anciallas = AncillaRegister(size = 4)
    qubits = QuantumRegister(size = 5)
    qc.add_register(anciallas)
    qc.add_register(qubits)

    state_preparation = prepare_circuit_initial_state(value, probability)
    qc.append(state_preparation, range(4,9))

    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)

    qc.cx(0, 4)
    qc.cz(1, 5, 'Z')

    qc.cz(0, 6, 'Z')
    qc.cx(1, 5)
    qc.cx(2, 4)

    qc.cx(7, 0)
    qc.cz(6, 1, 'Z')
    qc.cz(3, 4, 'Z')

    qc.cz(1, 7, 'Z')
    qc.cx(2, 6)
    qc.cx(3, 5)

    qc.cx(1, 8)
    qc.cz(2, 7, 'Z')

    qc.cz(2, 8, 'Z')
    qc.cx(3, 7)

    qc.cz(3, 8)

    qc.barrier()

    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)    

    # Measure ancillas to check what we need to correct
    error_correction(qc, anciallas)

    return qc

In [4]:
def error_correction(circuit, anciallas):
    syndromes = ClassicalRegister(4, name="syndromes")
    circuit.add_register(syndromes)
    circuit.measure(anciallas, syndromes)

    with circuit.switch(syndromes) as case:
        with case(0b0001):
            circuit.x(0)
        with case(0b1000):
            circuit.x(1)
        with case(0b1100):
            circuit.x(2)
        with case(0b0110):
            circuit.x(3)
        with case(0b0011):
            circuit.x(4)
        with case(0b1010):
            circuit.z(0)
        with case(0b0101):
            circuit.z(1)
        with case(0b0010):
            circuit.z(2)
        with case(0b1001):
            circuit.z(3)
        with case(0b0100):
            circuit.z(4)
        with case(0b1011):
            circuit.y(0)
        with case(0b1101):
            circuit.y(1)
        with case(0b1110):
            circuit.y(2)
        with case(0b1111):
            circuit.y(3)
        with case(0b0111):
            circuit.y(4)
            

In [5]:
circuit = five_qubit_error_code_circuit(True, 1)
circuit.draw()

»
       a0_0: »
             »
       a0_1: »
             »
       a0_2: »
             »
       a0_3: »
             »
       q1_0: »
             »
       q1_1: »
             »
       q1_2: »
             »
       q1_3: »
             »
       q1_4: »
             »
syndromes: 4/»
             »
«                                                                                   ┌───┐                                                                      »
«       a0_0: ──────────────────────────────────────────────────────────────────────┤ H ├──────────────────────────────────────────────────────────────────────»
«                                                                                   ├───┤                                                                      »
«       a0_1: ──────────────────────────────────────────────────────────────────────┤ H ├──────────────────────────────────────────────────────────────────────»
«                                                                                   ├───┤                                                                      »
«       a0_2: ──────────────────────────────────────────────────────────────────────┤ H ├──────────────────────────────────────────────────────────────────────»
«                                                                                   ├───┤                                                                      »
«       a0_3: ──────────────────────────────────────────────────────────────────────┤ H ├──────────────────────────────────────────────────────────────────────»
«             ┌─────────────────────────────────────────────────────────────────────┴───┴─────────────────────────────────────────────────────────────────────┐»
«       q1_0: ┤0                                                                                                                                              ├»
«             │                                                                                                                                               │»
«       q1_1: ┤1                                                                                                                                              ├»
«             │                                                                                                                                               │»
«       q1_2: ┤2 State Preparation(0,-0.25,-0.25,0,-0.25,0,0,-0.25,-0.25,0,0,0.25,0,0.25,-0.25,0,-0.25,0,0,-0.25,0,0.25,0.25,0,0,-0.25,0.25,0,-0.25,0,0,0.25) ├»
«             │                                                                                                                                               │»
«       q1_3: ┤3                                                                                                                                              ├»
«             │                                                                                                                                               │»
«       q1_4: ┤4                                                                                                                                              ├»
«             └───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘»
«syndromes: 4/═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════»
«                                                                                                                                                              »
«                      Z           ┌───┐                                      »
«       a0_0: ──■──────■───────────┤ X ├──────────────────────────────────────»
«               │   Z  │           └─┬─┘       Z                              »
«       a0_1: ──┼───■──┼───■─────────┼───■─────■─────────────■───────────────